### rankings_payoffs.ipynb

- Egalitarianism minimises the disadvantage of the worst off agent
    - Utilitarianism is the maximisation of total utility

- The purpose of rankings here is to remove the extent to which you prefer one solution to another

This notebook will
- Normalise the the rankings of a utilitarian solution in relation to an egalitarian one, and compare.

- Given an agents principles, create rankings based on how much they wish to be influenced by another agents rankings(?)
    - A preferred ranking of their preferences considering their ethical principle

- Once an agents ranking has been computed, we compare the distance between their rankings, and the rankings of each consensus (Egal, Util, Transition, HCVA) to find closest.

#### Full Explanation for clarity
Each agent is comprised of a value system, and a principle preference
- The value system has, for every value, a preference value of each value over every other value. These are our preferences.
    - The value system also has, for every action, an action judgement for every value. The action judgement tells us how much each value in the system would be promoted or demoted if that action is taken.
- Additionally, the principle preference tells us the extent to which the agent prefers a principle over another principle. As there are only 2 principles this is a number between 0-1. 0 signfies full support for Utilitarianism, and 1.0 signifies full support for egalitarianism

Here the principle preference isnt simply a vote for one thing or another, its just a distance measure. When every agents principle preferences are collated, we optimise on them to find HCVA. We could always use another metric, such as MSE of the principle preferences.

To convert these distances to rankings, we need to check each agents principle preference (a number between 0 and 1), convert that preference to an actual principle value (between 1.0 and 10.0), and then rank the 4 options based on their distance from the agents real preferred principle value.
- This removes the extent one solution is preferred over the other
- This gives an accurate ranking of each agents preference for each solution

The files chosen for use here are the processed_data from the European Social Study (1 action, 2 values) where the values for each nation have been multipled by a preference factor of 2.5 and an action factor of 5.0 giving a transition point of 1.40 and HCVA point of 2.1 

In [154]:
# import needed files/libs
import pandas as pd

agent_data_path = "/home/ia23938/Documents/GitHub/ValueSystemsAggregation/data/ess_example_data/single_example_results/single_example/08-01-2025-agent-data.csv"
agent_data_df = pd.read_csv(agent_data_path)

consensus_data_path_pref = "/home/ia23938/Documents/GitHub/ValueSystemsAggregation/data/ess_example_data/single_example_results/single_example/08-01-2025-actions.csv"
consensus_data_path_act = "/home/ia23938/Documents/GitHub/ValueSystemsAggregation/data/ess_example_data/single_example_results/single_example/08-01-2025-preferences.csv"
temp_pref = pd.read_csv(consensus_data_path_pref)
temp_act = pd.read_csv(consensus_data_path_act)
consensus_data_df = pd.merge(temp_pref, temp_act, on='p')

agent_principle_data_path = "/home/ia23938/Documents/GitHub/ValueSystemsAggregation/data/ess_example_data/single_example_results/single_example/13-01-2025-against-principles.csv"
agent_principle_data_df = pd.read_csv(agent_principle_data_path)
agent_principle_data_df.rename(columns={'rel': 'egal', 'nonrel': 'util'}, inplace=True)

#####
hcva = 5.5
test_case = "Bottom Quartile"
#####

In [155]:
# First create rankings of preferences for each consensus

consensus_rankings = {"util": 1.0, "egal": 10.0, "t": 1.3, "hcva": hcva}

In [156]:
for row in agent_principle_data_df.iterrows():
    index, row_data = row
    egal = row_data['egal']
    util = row_data['util']
    # Normalising means that egal is 0 and util is 1
    normalised_value = egal / (egal + util)
    agent_principle_data_df.at[index, 'normalised_value'] = normalised_value

# Now each row is normalised, we convert the normalised value into the principle value for each agent, between 1.0 and 10.0
for row in agent_principle_data_df.iterrows():
    index, row_data = row
    normalised_value = row_data['normalised_value']
    principle_value = normalised_value * 9 + 1
    agent_principle_data_df.at[index, 'principle_value'] = round(principle_value, 2)

# Now calculate the ranking for of consensus rankings for each principle we are interested in
for row in agent_principle_data_df.iterrows():
    index, row_data = row
    temp_distances = {}
    for principle, value in consensus_rankings.items():
        agent_principle = row_data['principle_value']
        temp_distances[principle] = abs(agent_principle - value)
    temp_distances = {key: value for key, value in sorted(temp_distances.items(), key=lambda item: item[1])}
    # Place in dataframe for agent
    for i, (principle, distance) in enumerate(temp_distances.items()):
        agent_principle_data_df.at[index, principle + "_rank"] = i + 1

agent_principle_data_df.head()


,country,egal,util,normalised_value,principle_value,egal_rank,hcva_rank,t_rank,util_rank
0,AT,1449.0,409.0,0.779871,8.02,1.0,2.0,3.0,4.0
1,BE,191.0,1546.0,0.109960,1.99,4.0,3.0,1.0,2.0
2,CH,1003.0,430.0,0.699930,7.30,2.0,1.0,3.0,4.0
3,CZ,1978.0,82.0,0.960194,9.64,1.0,2.0,3.0,4.0
4,DE,1486.0,1266.0,0.539971,5.86,2.0,1.0,3.0,4.0


### Display results in LaTeX format

In [157]:
# Calculate the frequency of each strategy at each rank
rank_frequencies = agent_principle_data_df[['hcva_rank', 't_rank', 'util_rank', 'egal_rank']].apply(pd.Series.value_counts).fillna(0).astype(int)

# Create a LaTeX table
latex_rank_frequencies = rank_frequencies.T.to_latex()

latex_table_with_caption = "\\begin{table}[htb]\n\\centering\n\\caption{Ranking of each strategy where \method is based on "+ test_case+" synthetic principles }\n" + latex_rank_frequencies + "\n\\end{table}"

print(latex_table_with_caption)

\begin{table}[htb]
\centering
\caption{Ranking of each strategy where \method is based on Bottom Quartile synthetic principles }
\begin{tabular}{lrrrr}
\toprule
 & 1.000000 & 2.000000 & 3.000000 & 4.000000 \\
\midrule
hcva_rank & 6 & 6 & 11 & 0 \\
t_rank & 11 & 2 & 10 & 0 \\
util_rank & 0 & 11 & 2 & 10 \\
egal_rank & 6 & 4 & 0 & 13 \\
\bottomrule
\end{tabular}

\end{table}


In [158]:
# Sum up the ranks for each principle
rank_sums = agent_principle_data_df[['hcva_rank', 't_rank', 'util_rank', 'egal_rank']].sum()

# Create a LaTeX table
latex_table = rank_sums.to_frame(name='Sum of Ranks (Lower is better)').T.to_latex()

print(latex_table)

\begin{tabular}{lrrrr}
\toprule
 & hcva_rank & t_rank & util_rank & egal_rank \\
\midrule
Sum of Ranks (Lower is better) & 51.000000 & 45.000000 & 68.000000 & 66.000000 \\
\bottomrule
\end{tabular}



In [159]:
agent_data_df

,country,rel,nonrel,Rel-Nonrel,Nonrel-Rel,a_div_rel,a_div_nonrel
0,AT,642,1216,0.345533,0.654467,0.047767,0.112390
1,BE,621,1116,0.357513,0.642487,-0.063875,-0.054361
2,CH,767,666,0.535241,0.464759,0.150804,0.262262
3,CZ,577,1483,0.280097,0.719903,0.021375,0.028995
4,DE,1264,1488,0.459302,0.540698,0.002110,0.068100
5,EE,802,1118,0.417708,0.582292,0.072319,0.049493
6,ES,567,1143,0.331579,0.668421,0.014697,0.056285
7,FI,739,1099,0.402067,0.597933,-0.098782,-0.005763
8,FR,1063,899,0.541794,0.458206,0.021010,0.112347
9,GB,541,1287,0.295952,0.704048,0.008010,0.050505


#### Consider the principle value system, rather than the distance away from the principle value itself

Repeat the same code again, formatting dataframes for comaprision

In [160]:
agent_data_df = agent_data_df[['country','Rel-Nonrel','Nonrel-Rel', 'a_div_rel', 'a_div_nonrel']]
agent_principle_data_df = agent_principle_data_df[['country','principle_value']]
agent_data_df = pd.merge(agent_data_df, agent_principle_data_df, on='country')
agent_data_df.rename(columns={'principle_value': 'p', 'a_div_rel' : 'Rel_div_p', 'a_div_nonrel': 'Nonrel_div_p'}, inplace=True)

In [161]:
consensus_data_path_pref = "/home/ia23938/Documents/GitHub/ValueSystemsAggregation/data/ess_example_data/single_example_results/factored_single_example/02-01-2025-actions-a-2.5-p-5.0.csv"
consensus_data_path_act = "/home/ia23938/Documents/GitHub/ValueSystemsAggregation/data/ess_example_data/single_example_results/factored_single_example/02-01-2025-preferences-a-2.5-p-5.0.csv"
temp_pref = pd.read_csv(consensus_data_path_pref)
temp_act = pd.read_csv(consensus_data_path_act)
consensus_data_df = pd.merge(temp_pref, temp_act, on='p')
consensus_data_df = consensus_data_df[['p','Rel_div_p', 'Nonrel_div_p', 'Rel-Nonrel', 'Nonrel-Rel']]


#### Normalising the values for comparison

The values can't just be the distance between each one, as the scale between each value is different

In [162]:
# Normalise all data between 0-1
consensus_data_df = consensus_data_df.astype(float)
unnormalised_consensus_data_df = consensus_data_df.copy()
for row in consensus_data_df.iterrows():
    for column in consensus_data_df.columns:
        min_val = consensus_data_df[column].min()
        max_val = consensus_data_df[column].max()
        if column == 'p':
            consensus_data_df['p_normalised'] = consensus_data_df['p']
            column = 'p_normalised'
        consensus_data_df[column] = (consensus_data_df[column] - min_val) / (max_val - min_val)

# Filter consensus_data_df to keep only rows with p values of 1.0 1.4, 2.1, 10.0
relevant_p_values = [1.0, 1.4, hcva, 10.0]
tolerance = 1e-5
consensus_data_df = consensus_data_df[consensus_data_df['p'].apply(lambda x: any(abs(x - val) < tolerance for val in relevant_p_values))]


# Now the same for agents, but work from the unnormalised consensus data
for row in agent_data_df.iterrows():
    for column in agent_data_df.columns:
        if column == 'country' or column == 'p_normalised':
            continue
        min_val = agent_data_df[column].min()
        max_val = agent_data_df[column].max()
        if column == 'p':
            agent_data_df['p_normalised'] = agent_data_df['p']
            column = 'p_normalised'
        agent_data_df[column] = (agent_data_df[column] - min_val) / (max_val - min_val)


In [163]:
# Now calculate the ranking for of consensus rankings for each principle we are interested in \ 'p_normalised'
comparison_data = ['Rel_div_p', 'Nonrel_div_p', 'Rel-Nonrel', 'Nonrel-Rel']
for agent_row in agent_data_df.iterrows():
    #print(agent_row)
    temp_strategy_distances = {}
    temp_distances = []
    for consensus_row in consensus_data_df.iterrows():
        #print(consensus_row)
        for column in comparison_data:
            agent_value = agent_row[1][column]
            consensus_value = consensus_row[1][column]
            abs_difference = abs(agent_value - consensus_value)
            #print(f"Column: {column} Agent value: {agent_value}, Consensus value: {consensus_value}, Absolute difference: {abs_difference}")
            temp_distances.append(abs_difference)
        temp_strategy_distances[consensus_row[1]['p']] = sum(temp_distances)
        temp_distances = []
    strategy_temp_distances = {key: value for key, value in sorted(temp_strategy_distances.items(), key=lambda item: item[1])}
    #print(strategy_temp_distances)
    # Place in dataframe for agent
    for i, (principle, distance) in enumerate(strategy_temp_distances.items()):
        #print(f"Principle: {principle}, Distance: {distance}")
        agent_data_df.at[agent_row[0], f'{principle}_rank'] = i + 1

agent_data_df.rename(columns={'1.0_rank' : 'util_rank','5.499999999999997_rank' : 'hcva_rank', '1.4000000000000004_rank' : 't_rank', '9.999999999999982_rank' : 'egal_rank'}, inplace=True)

In [164]:
agent_data_df.head()

,country,Rel-Nonrel,Nonrel-Rel,Rel_div_p,Nonrel_div_p,p,p_normalised,util_rank,t_rank,5.499999999999997_rank,egal_rank
0,AT,0.437081,0.562919,0.677556,0.664863,8.02,0.788235,1.0,2.0,3.0,4.0
1,BE,0.459931,0.540069,0.483986,0.399598,1.99,0.000000,4.0,2.0,1.0,3.0
2,CH,0.798920,0.201080,0.856204,0.903278,7.30,0.694118,3.0,4.0,1.0,2.0
3,CZ,0.312272,0.687728,0.631796,0.532199,9.64,1.000000,2.0,1.0,3.0,4.0
4,DE,0.654079,0.345921,0.598393,0.594407,5.86,0.505882,4.0,3.0,1.0,2.0


In [165]:
consensus_data_df

,p,Rel_div_p,Nonrel_div_p,Rel-Nonrel,Nonrel-Rel,p_normalised
0,1.0,1.000000,1.000000,0.027241,0.972759,0.000000
4,1.4,0.212419,0.512429,0.003103,0.996897,0.043956
45,5.5,0.333040,0.000558,0.788848,0.211152,0.494505
90,10.0,0.583467,0.005097,0.996726,0.003274,0.989011


#### Describe ranks in LaTeX
Here we can see that the most common rank is 1st for HCVA, with the Transition point performing second best in this random society. By summing the ranks, HCVA is best also. This maps to the previous data, which compares only the principles 

In [166]:
# Calculate the frequency of each strategy at each rank
rank_frequencies = agent_data_df[['hcva_rank', 't_rank', 'util_rank', 'egal_rank']].apply(pd.Series.value_counts).fillna(0).astype(int)

# Create a LaTeX table
latex_rank_frequencies = rank_frequencies.T.to_latex()

print(latex_rank_frequencies)

KeyError: "['hcva_rank'] not in index"

In [152]:
# Sum up the ranks for each principle
rank_sums = agent_data_df[['hcva_rank', 't_rank', 'util_rank', 'egal_rank']].sum()

# Create a LaTeX table
latex_table = rank_sums.to_frame(name='Sum of Ranks (Lower is better)').T.to_latex()

print(latex_table)

\begin{tabular}{lrrrr}
\toprule
 & hcva_rank & t_rank & util_rank & egal_rank \\
\midrule
Sum of Ranks (Lower is better) & 46.000000 & 46.000000 & 69.000000 & 69.000000 \\
\bottomrule
\end{tabular}



#### Borda Count Analysis of results
see https://en.wikipedia.org/wiki/Borda_count for details

In [153]:
# Assign Borda points to each rank
borda_points = {1: 3, 2: 2, 3: 1, 4: 0}

# Initialize a dictionary to store the Borda scores for each principle
borda_scores = {'util': 0, 'hcva': 0, 't': 0, 'egal': 0}

# Sum the Borda points for each principle
for index, row in agent_data_df.iterrows():
    borda_scores['util'] += borda_points[row['util_rank']]
    borda_scores['hcva'] += borda_points[row['hcva_rank']]
    borda_scores['t'] += borda_points[row['t_rank']]
    borda_scores['egal'] += borda_points[row['egal_rank']]

# Determine the principle with the highest Borda score
winner = max(borda_scores, key=borda_scores.get)

print("Borda Scores:", borda_scores)
print("Winner:", winner)
# Create a LaTeX table for the Borda scores
borda_scores_df = pd.DataFrame(list(borda_scores.items()), columns=['Principle', 'Borda Score'])
latex_borda_scores = borda_scores_df.to_latex(index=False)

print(latex_borda_scores)

Borda Scores: {'util': 23, 'hcva': 46, 't': 46, 'egal': 23}
Winner: hcva
\begin{tabular}{lr}
\toprule
Principle & Borda Score \\
\midrule
util & 23 \\
hcva & 46 \\
t & 46 \\
egal & 23 \\
\bottomrule
\end{tabular}

